# EMA Calculation Script VENSIM Model

Date: 28-08-2017

#Mark Havelaar
#Wouter Jaspers

Subject: Improving Large Intervention Decisions within Infra Asset Management

Model: FinalModelEMA

DUO Simulation_RoadLockIntervention

#1 Loading EMA Workbench & Other Relevant Packages

In [2]:
import pandas as pd

import numpy as np

import matplotlib.pyplot as plt

from __future__ import (division, unicode_literals, print_function, 
                        absolute_import)

from ema_workbench import (TimeSeriesOutcome, perform_experiments, Constant, 
                           RealParameter, ema_logging, ScalarOutcome, load_results)

from ema_workbench.em_framework.evaluators import MultiprocessingEvaluator

from ema_workbench.util import save_results, ema_logging, CaseError

from ema_workbench.analysis.plotting import envelopes 
from ema_workbench.analysis.plotting_util import (KDE, BOXPLOT)

from ema_workbench.analysis.plotting import lines

from ema_workbench.analysis.pairs_plotting import (pairs_lines, pairs_scatter)

from ema_workbench.connectors.vensim import (LookupUncertainty, VensimModel, VensimModelStructureInterface)

from ema_workbench.em_framework.parameters import Policy

from ema_workbench.em_framework import CategoricalParameter

#Turn on Logging
ema_logging.log_to_stderr(ema_logging.INFO)

<Logger EMA (DEBUG)>

#2  Loading VENSIM Model

In [3]:
#Specify Location
wd= r'C:\Users\markh\Dropbox\Tu Delft\Afstuderen SD\V - EMA'

In [4]:
#Load Model
if __name__ == '__main__':
    ema_logging.log_to_stderr(ema_logging.INFO)

model = VensimModel("EMAModel", wd=wd, model_file= wd + '\FinalModelEMA.vpm')

#3 Specifying Model Uncertainties for all Simulations

In [5]:
##SCENARIO'S

model.uncertainties=[   
    
#CONNECTED/UNCONNECTED
                            CategoricalParameter('Switch Connected and Unconnected', (0,1)),
#Population Scenario
                            CategoricalParameter('Switch Population', (0,1,2)),
#Land Use Scenario
                            CategoricalParameter('Switch Urbanization', (0,1,2)),
#Economy Scenario
                            CategoricalParameter('Switch Economy', (0,1,2)),
#Innovation Scenario
                            CategoricalParameter('Switch Innovation', (0,1,2)),
#Climate Scenario
                            CategoricalParameter('Switch Climate Scenario', (0,1,2,3)),
                       
##GENERAL UNCERTAINTIES
                            
#Building Mode City and Harbour                   
                            CategoricalParameter('Switch Building Mode', (0,1)),
                            CategoricalParameter('Forecast Time Forecasting Policy', (1, 2, 3, 4)),
    
#Nature Preservation Area
                            CategoricalParameter('Switch Nature Scenario', (0,1)),
                            
#MODEL SPECIFIC UNCERTAINTIES
                            
#City Model
                            RealParameter('Normal Land use per house', 0.024, 0.036),
                            CategoricalParameter('Forecasting Delay Order', (1,3,5)),
                            CategoricalParameter('Planning Time Houses', (1, 2, 3, 4, 5)),
                            CategoricalParameter('Construction Time Houses', (1, 2, 3)),
                            CategoricalParameter('Delay Order House Construction', (1,3,5)),
                            RealParameter('Technical Lifetime Houses', 300, 500),
                            RealParameter('Normal Land Use per Business', 0.048, 0.072),
                            RealParameter('Jobs per harbour mtpa', 100, 150),
                            CategoricalParameter('Planning Time Business', (1, 2, 3, 4, 5)),
                            CategoricalParameter('Construction Time Business', (1,2, 3)),
                            CategoricalParameter('Delay Order Business Construction', (1,3,5)),
                            RealParameter('Technical Lifetime Business', 150, 250),
#Harbour Model
                            CategoricalParameter('Planning Time Harbour', (1,2,3,4,5)),
                            CategoricalParameter('Construction Time Harbour', (1,2,3)),
                            CategoricalParameter('Delay Order Harbour Construction', (1,3,5)),
                            RealParameter('Technical Lifetime Harbour', 150, 250),
#Area Model
                            ## No Model Uncertainties
#Economy Model
                            ## No Model Uncertainties
#Road Model
                            CategoricalParameter('Planning Time New Construction Road', (5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15)),
                            CategoricalParameter('Planning Time Renovation Road', (1, 2, 3)),
                            CategoricalParameter('Renovation Time Road', (0.25, 0.5, 0.75, 1)),
                            CategoricalParameter('Construction Time Road', (1, 2, 3, 4, 5)),
                            RealParameter('Closure Rate Road A10 Intervention', 0.33, 0.50),
                            RealParameter('PCE to MVT factor', 0.5, 2),
                            CategoricalParameter('Delay Time Congestion Divergence', (0.5, 0.75, 1)),
                            RealParameter('MTPA to Truck Factor', 0.000004, 0.000006),
    
#Lock Model
                            CategoricalParameter('Delay time divergence factor', (0.5, 1.0, 1.5)),
                            CategoricalParameter('Delay time Scaling Benefits', (0.5, 1.0, 1.5)),
                            CategoricalParameter('Construction Time New Sea Lock', (2, 3, 4)),
                            CategoricalParameter('Planning Time ReOpening old Noordersluis', (2, 3, 4)),
                            CategoricalParameter('Construction Time Reopening old Noordersluis', (2, 3, 4)),
                            CategoricalParameter('Construction Time New Sea Lock No Innovation', (2, 3, 4)),
                            CategoricalParameter('Planning Time New Sea Lock No Innovation', (2, 3, 4)),
                            CategoricalParameter('Planning Time New Sea Lock Innovation', (3, 4, 5, 6)),
                            CategoricalParameter('Construction Time New Sea Lock Innovation', (3, 4, 5, 6)),
                            CategoricalParameter('Initial MTPA Leisure Vessels', (2, 3, 4))
                            ]                          

#4 Specifying Model Constants for all Simulations

In [16]:
##GENERAL MODEL CONSTANTS
model.constants = [
#City Model
                        Constant('Initial Planned Houses', 5000),
                        Constant('Initial Houses', 423785), 
                        Constant('Normal size households', 1.97),
                        Constant('Initial population', 834713),
                        Constant('Labour population factor', 0.8),
                        Constant('Initial Number of jobs per business unit', 4.64),
                        Constant('Initial Planned Business', 1000),
                        Constant('Initial Business', 100000),
                        Constant('Percentage of Labour Population actively wanting a job', 0.7),
                        Constant('Labour population out of City factor', 0.07),
                
#Harbour Model
                        Constant('Initial Planned Harbour Capacity', 50),
                        Constant('Initial Constructed Harbour Capacity', 200),
                        Constant('MTPA Canal Capacity', 500),
                        Constant('MTPA Train Capacity', 50),
                        Constant('Initial world offer sea MTPA', 95),
                        Constant('Initial world offer road MTPA', 38),
                        Constant('Initial world offer rail MTPA', 5),
                        Constant('Initial world offer canal MTPA', 63),
                        Constant('Percentage Harbour Distribution Capacity Through Sea', 0.472637),
                        Constant('Percentage Harbour Distribution Capacity Through Road', 0.189055),
                        Constant('Percentage Harbour Distribution Capacity Through Canal', 0.313433),
                        Constant('Percentage Harbour Distribution Capacity Through Train', 0.0248756),
#Area Model
                        Constant('Initial Free Area', 25000),
                        Constant('Rail Planned', 0),
                        Constant('Canal Planned', 0),
                        Constant('Lock Planning', 0),
                        Constant('Initial Nature Area', 7500),
                        Constant('Initial Lock Area', 6),
                        Constant('Initial Canal Area', 1500),
                        Constant('Initial Rail Area', 100),
                        Constant('Initial Flooding Area', 1500),
                        Constant('Rail Demolished', 0),
                        Constant('Lock Demolition', 0),
                        Constant('Canal Demolished', 0),
                        Constant('Road Lanes Demolished', 0),
                        Constant('Flooding Area Demolished', 0),
#Economy Model
                        Constant('Initial Local Economy', 62603475000),
                        Constant('Initial Shares Business', 0.25),
                        Constant('Initial Shares Harbour', 0.25),
                        Constant('Initial Shares Job Spending', 0.25),
#Road Model
                        Constant('Number of Weakest Links', 1),
                        Constant('Number of Distribution Roads', 5),   
                        Constant('Initial Planned Road Capacity', 0),
                        Constant('Initial Deteriorated Road Capacity', 0),
                        Constant('Initial Diminshed Road Capacity', 0),
                        Constant('Normal Truck Percentage for calculating congestion limit', 0.15),
                        Constant('Rush Hour Car Intensity', 9000),
                        Constant('Initial IC Factor Road Intervention Moment Memory', 0),
                        Constant('Initial Intervention Moment Sewer', 0),
                        Constant('Initial Sewer System Maximum Capacity', 45),
                        Constant('Sewer Capacity increased After Intervention', 55),
                        Constant('Free Flow Capacity per Lane A10', 2000),
                        Constant('Free Flow Capacity per Lane A8', 2100),
                        Constant('Free Flow Capacity per Lane A2', 2100),
                        Constant('Free Flow Capacity per Lane A4', 2100),
                        Constant('Free Flow Capacity per Lane A5', 2100),
                        Constant('Area per Road Lane', 15),
                        Constant('Initial Number of Lanes A2', 8),
                        Constant('Initial Number of Lanes A4', 8),
                        Constant('Initial Number of Lanes A5', 4),
                        Constant('Initial Number of Lanes A10', 6),
                        Constant('Initial Number of Lanes A8', 10),
                        Constant('Delay Order Divergence', 5),
                        Constant('Reference Maximum Precipitation', 0.00414),
#Lock Model
                        Constant('Initial Cumulative Deteriorated Lock Capacity', 0),
                        Constant('Initial Constructed Capacity New Sea Lock', 0),
                        Constant('Initial Capacity Old Middensluis', 25),
                        Constant('Initial Capacity Old Noordersluis', 66.667),
                        Constant('Initial Capacity Kleine Sluis and Zuidersluis', 8.33),
                        Constant('Initial Planned Lock Capacity', 0),
                        Constant('Demolishing Rate New Locks', 0),
                        Constant('Factor for increase required door height', 0.75833),
                        Constant('Initial Required Height for doors', 515),
                        Constant('Initial Required Height for remaining parts lock', 700),
                        Constant('Factor for increase required height remaining parts lock', 0.75833),
                        Constant('Minimum present Height of remaining parts', 725),
                        Constant('Height door Old Noordersluis', 585),
                        Constant('Height door Old Middensluis', 585),
                        Constant('IC Lock Decision Making Factor', 0.5),
                        Constant('Initial Door Height Intervention Moment Memory', 0),
                        Constant('Initial Remaining Parts Height Intervention Moment Memory', 0),
                        Constant('Initial IC Factor Lock Intervention Moment Memory', 0),
                        Constant('MTPA Increase by New Sea Lock', 175),
                        Constant('Delay Order Divergence factor', 5),
                        Constant('Delay order Scaling benefits', 5),  
       
## SIMULATION SETTINGS
                #Road Model
                        Constant('Number of Lanes Added', 2),
                        Constant('IC Decision Making Factor Road', 0.8),
                        Constant('Switch Intervention Policy Road', 5),
                        Constant('Policy Combination Switch', 3),
                        Constant('Switch Sewer Intervention', 1),
                        Constant('Base Case Intervention Moment', 0),
                        Constant('First Intervention Moment Road System', 2017),
                        Constant('Second Intervention Moment Road System', 2019), 
                #Lock Model
                        Constant('Switch Lock Policy', 5),
                        Constant('Intervention Moment New Sea Lock', 2017),
                        Constant('Intervention Moment for 2nd New Sea Lock No Innovation', 0),
                        Constant('Intervention Moment for 2nd New Sea Lock Innovation', 0),
                        Constant('Intervention Moment for 3rd New Sea Lock No Innovation', 0),
                        Constant('Intervention Moment for 3rd New Sea Lock Innovation', 2026),
                        ]

#6 Specifying Model Key Performance Indicators (KPIs)

In [7]:
#Important Model Outcomes
model.outcomes = [
                 TimeSeriesOutcome('City Population'),
                 ScalarOutcome('MTPA Distributed in Harbour'),
                 ScalarOutcome('Available Area'),
                 TimeSeriesOutcome('Local Economy'),
                 ScalarOutcome('Local Economic Growth'),
                 ScalarOutcome('Road System IC Factor'),
                 ScalarOutcome('Lock System IC Factor'),
                 ]

#7 Perform Experiments + Lines Plots

In [17]:
#Perform Experiments
nr_experiments = 1000

results = perform_experiments(model, nr_experiments)

[MainProcess/INFO] performing 1000 scenarios * 1 policies * 1 model(s) = 1000 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 100 cases completed
[MainProcess/INFO] 200 cases completed
[MainProcess/INFO] 300 cases completed
[MainProcess/INFO] 400 cases completed
[MainProcess/INFO] 500 cases completed
[MainProcess/INFO] 600 cases completed
[MainProcess/INFO] 700 cases completed
[MainProcess/INFO] 800 cases completed
[MainProcess/INFO] 900 cases completed
[MainProcess/INFO] 1000 cases completed
[MainProcess/INFO] experiments finished


#8 Saving Results

In [18]:
save_results(results, wd + '\\DUO_1000runs_SimulationFile_RoadXLockX.tar.gz')

[MainProcess/INFO] results saved successfully to C:\Users\markh\Dropbox\Tu Delft\Afstuderen SD\V - EMA\DUO_1000runs_SimulationFile_RoadXLockX.tar.gz
